# Feinschmecker

In [102]:
import owlready2
import rdflib
from owlready2.ntriples_diff import Blank
# imports
from rdflib import Graph, Literal, URIRef, Namespace
from rdflib.namespace import OWL, RDF, RDFS, XSD
from owlready2 import *


### Factory creations

In [103]:
NAMESPACE = "https://jaron@sprute.com/uni/actionable-knowledge-representation/feinschmecker"
onto = get_ontology(NAMESPACE)

def ThingFactory(name, BaseClass=Thing) -> type[Thing]:
    with onto:
        return type[Thing](name, (BaseClass,), {})

def RelationFactory(name, domain:list[ThingClass]=None, range=None) -> type[ObjectProperty]:
    if domain is None:
        domain = [Thing]
    if range is None:
        range = [Thing]
    with onto:
        return type[ObjectProperty](name, (ObjectProperty, ), {
            "domain": domain,
            "range": range,
        })

def DataFactory(name, domain:list[ThingClass]=None, range=None) -> type[DataProperty]:
    if domain is None:
        domain = [Thing]
    if range is None:
        range = [OWL.Nothing]
    with onto:
        return type[DataProperty](name, (DataProperty, ), {
            "domain": domain,
            "range": range,
        })

def makeInverse(first: ObjectProperty, second: ObjectProperty) -> None:
    if first is None or second is None:
        raise TypeError("There is no inverse of no element: first:", str(first) , "second:", str(second))
    with onto:
        first.inverse_property = second
        second.inverse_property = first


### Class creations

In [104]:
Recipe = ThingFactory("Recipe")
Ingredient = ThingFactory("Ingredient")
Author = ThingFactory("Author")
Source = ThingFactory("Source")
Time = ThingFactory("Time")

Concepts = ThingFactory("Concepts")
Vegetarian = ThingFactory("Vegetarian", BaseClass=Concepts)
Vegan = ThingFactory("Vegan", BaseClass=Concepts)
Difficulty = ThingFactory("Difficulty", BaseClass=Concepts)

Calories = ThingFactory("Calories")
Protein = ThingFactory("Protein", BaseClass=Calories)
Fat = ThingFactory("Fat", BaseClass=Calories)
Carbohydrates = ThingFactory("Carbohydrates", BaseClass=Calories)

### Relation creations

In [105]:
# Descends according to the class creations above

# Recipe
has_ingredient = RelationFactory("has_ingredient", domain=[Recipe], range=[Ingredient])
authored_by = RelationFactory("authored_by", domain=[Recipe], range=[Author])
requires_time = RelationFactory("requires_time", domain=[Recipe], range=[Time])

is_vegan = RelationFactory("is_vegan", domain=[Recipe], range=[Vegan])
is_vegetarian = RelationFactory("is_vegetarian", domain=[Recipe], range=[Vegetarian])
has_difficulty = RelationFactory("has_difficulty", domain=[Recipe], range=[Difficulty])

has_calories = RelationFactory("has_calories", domain=[Recipe], range=[Calories])
has_protein = RelationFactory("has_protein", domain=[Recipe], range=[Protein])
has_fat = RelationFactory("has_fat", domain=[Recipe], range=[Fat])
has_carbohydrates = RelationFactory("has_carbohydrates", domain=[Recipe], range=[Carbohydrates])

has_link = DataFactory("has_link", domain=[Recipe], range=[str]) # RDF.HTML is not supported, see https://owlready2.readthedocs.io/en/latest/properties.html#data-property 

# Ingredient
is_ingredient_of = RelationFactory("is_ingredient_of", domain=[Ingredient], range=[Recipe])

# Author
authored = RelationFactory("authored", domain=[Author], range=[Recipe])
is_user_of = RelationFactory("is_user_of", domain=[Author], range=[Source])

# Source
has_user = RelationFactory("has_user", domain=[Source], range=[Author])

# Time
time_required = RelationFactory("time_required", domain=[Time], range=[Recipe])

# Concepts
vegan_applies_to = RelationFactory("vegan_applies_to", domain=[Vegan], range=[Recipe])
vegetarian_applies_to = RelationFactory("vegetarian_applies_to", domain=[Vegetarian], range=[Recipe])
difficulty_applies_to = RelationFactory("difficulty_applies_to", domain=[Difficulty], range=[Recipe])

# Calories
calories_of = RelationFactory("calories_of", domain=[Calories], range=[Recipe])
protein_of = RelationFactory("protein_of", domain=[Protein], range=[Recipe])
fat_of = RelationFactory("fat_of", domain=[Fat], range=[Recipe])
carbohydrates_of = RelationFactory("carbohydrates_of" , domain=[Carbohydrates], range=[Recipe])




### Interrelational properties

In [106]:
# Inverses
makeInverse(has_ingredient, is_ingredient_of)
makeInverse(authored_by, authored)
makeInverse(is_vegan, vegan_applies_to)
makeInverse(is_vegetarian, vegetarian_applies_to)
makeInverse(has_difficulty, difficulty_applies_to)
makeInverse(is_user_of, has_user)
makeInverse(requires_time, time_required)
makeInverse(has_calories, calories_of)
makeInverse(has_protein, protein_of)
makeInverse(has_fat, fat_of)
makeInverse(has_carbohydrates, carbohydrates_of)

# Limitations
with onto:
    # Recipe
    Recipe.is_a.append(has_ingredient.exactly(1, Ingredient))
    Recipe.is_a.append(authored_by.exactly(1, Author))
    Recipe.is_a.append(is_vegan.max(1, Vegan))
    Recipe.is_a.append(is_vegetarian.max(1, Vegetarian))
    Recipe.is_a.append(has_difficulty.max(1, Difficulty)) # Debatable
    Recipe.is_a.append(has_calories.max(1, Calories)) # Debatable
    Recipe.is_a.append(has_protein.exactly(1, Protein))
    Recipe.is_a.append(has_fat.exactly(1, Fat))
    Recipe.is_a.append(has_carbohydrates.exactly(1, Carbohydrates))
    Recipe.is_a.append(has_link.exactly(1, str))
    
    # Ingredient
    Ingredient.is_a.append(is_ingredient_of.some(Recipe))
    
    # Author
    Author.is_a.append(is_user_of.exactly(1, Source))
    
    # Source
    
    # Concepts
    
    # Calories

# Disjointness
with onto:
    AllDisjoint([Recipe, Ingredient, Author, Source, Concepts, Calories])

### Individuals

In [107]:
recipeTest = Recipe("recipeTest")
ingredientTest = Ingredient("ingredientTest")

recipeTest.has_ingredient.append(ingredientTest)
print(ingredientTest.is_ingredient_of)


[feinschmecker.recipeTest]


In [108]:
with onto:
    print(list(default_world.individuals()))
    print(list(default_world.inconsistent_classes()))

[feinschmecker.recipeTest, feinschmecker.ingredientTest]
[]


## Temporary deletion to avoid side effects

In [109]:
onto.save("feinschmecker.rdf")
onto.destroy(update_relation=True, update_is_a=True)